In [2]:
# %load notebooks/utils/get_dataframes.py
import pandas as pd

DATA_DIR = 'data'
MUTANTS_CONTEXT = '{0}/{1}/mutants.context'
KILL_CSV = '{0}/{1}/kill.csv'


def get_subject(subject):

    mutants_context_df = pd.read_csv(MUTANTS_CONTEXT.format(DATA_DIR, subject))
    kill_df = pd.read_csv(KILL_CSV.format(DATA_DIR, subject))

    del mutants_context_df['mutationOperatorGroup']
    del mutants_context_df['nodeTypeDetailed']
    del mutants_context_df['nodeContextBasic']
    del mutants_context_df['astContextDetailed']
    del mutants_context_df['parentContextBasic']
    del mutants_context_df['parentContextDetailed']
    del mutants_context_df['parentStmtContextBasic']
    del mutants_context_df['parentStmtContextDetailed']

    kill = []

    for i in mutants_context_df['mutantNo']:

        search = kill_df.query('MutantNo == %d' % i)

        if len(search) > 0:
            kill.append(search.iloc[0, 1])
        else:
            kill.append(None)

    mutants_context_df['kill'] = kill

    return mutants_context_df.dropna(axis=0, how='any').reset_index(drop=True)


def get_subjects(subjects):
    mutant_context_df = None

    for subject in subjects:
        if mutant_context_df is not None:
            mutant_context_df = mutant_context_df.append(
                get_subject(subject))
        else:
            mutant_context_df = get_subject(subject)

    return mutant_context_df.reset_index(drop=True)



In [3]:

# %load notebooks/utils/get_x.py
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


def get_x(df):
    vectorizer_ast = CountVectorizer(ngram_range=(1, 4), analyzer='word')
    ast_vectorized = vectorizer_ast.fit_transform(
        df['astContextBasic'])

    vectorizer_mutation_op = CountVectorizer()
    mutation_op_vectorized = vectorizer_mutation_op.fit_transform(
        df['mutationOperator'])

    vectorizer_node_type = CountVectorizer()
    node_type_vectorized = vectorizer_node_type.fit_transform(
        df['nodeTypeBasic'])

    ast_df = pd.DataFrame(
        ast_vectorized.A,
        columns=vectorizer_ast.get_feature_names())

    mutation_op_df = pd.DataFrame(
        mutation_op_vectorized.A,
        columns=vectorizer_mutation_op.get_feature_names())

    node_type_df = pd.DataFrame(
        node_type_vectorized.A,
        columns=vectorizer_node_type.get_feature_names())

    x = df.iloc[:, 4:7]
    x = x.join(ast_df.add_prefix('ast_'))
    x = x.join(mutation_op_df.add_prefix('mop_'))
    x = x.join(node_type_df.add_prefix('nty_'))

    return x

In [4]:
# %load notebooks/utils/get_y.py
def get_y(df, column):
    status_to_nonequivalent = {
        'FAIL': 'NON_EQUIVALENT',
        'TIME': 'NON_EQUIVALENT',
        'EXC': 'NON_EQUIVALENT',
        'LIVE': 'MAYBE_EQUIVALENT'
    }

    status_to_trivial = {
        'FAIL': 'NON_TRIVIAL',
        'TIME': 'NON_TRIVIAL',
        'EXC': 'TRIVIAL',
        'LIVE': 'NON_TRIVIAL'
    }

    df['non_equivalent'] = df['kill'].str.upper().map(status_to_nonequivalent)
    df['trivial'] = df['kill'].str.upper().map(status_to_trivial)

    return df[column]

In [5]:
# %load notebooks/utils/metrics.py
from sklearn import metrics


def measure_performance(X, y, clf):
    y_predicted = clf.predict(X)
    print('Accuracy: %f \n' % metrics.accuracy_score(y, y_predicted))
    print(metrics.classification_report(y, y_predicted), '\n')

In [6]:
# %load notebooks/utils/samples.py
def get_samples(df, label, frac=0.1, replace=True):
    X = get_x(df)
    X['y'] = get_y(df, label)

    X_train = X.sample(frac=frac, replace=replace)

    y = X['y']
    y_train = X_train['y']

    X = X.drop(['y'], axis=1)
    X_train = X_train.drop(['y'], axis=1)

    return X, y, X_train, y_train


In [7]:
from sklearn.naive_bayes import BernoulliNB

label = 'non_equivalent'

subjects = [
    'Closure-5', 'Closure-8', 'Closure-12'
]


mutant_context_df = get_subjects(subjects)

X, y, X_train, y_train = get_samples(mutant_context_df, label, frac=1)


clf = BernoulliNB()

clf = clf.fit(X_train, y_train)


measure_performance(X, y, clf)

Accuracy: 0.745399 

                  precision    recall  f1-score   support

MAYBE_EQUIVALENT       0.44      0.54      0.48        72
  NON_EQUIVALENT       0.86      0.80      0.83       254

     avg / total       0.77      0.75      0.75       326
 

